In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tensorboardX


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
import torch, torchvision
import torch.nn as nn

# from torchvision import models, transforms, datasets
import torchvision.models as models
import torchvision.datasets as datasets
from torchvision import transforms

import torch.optim as optim
from tqdm import tqdm  # tqdm 라이브러리 임포트
from tensorboardX import SummaryWriter  # TensorBoardX 임포트

import matplotlib.pyplot as plt
from PIL import Image

Deep Residual Learning for Image Recognition

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1512.03385


Identity Mappings in Deep Residual Networks

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1603.05027

In [ ]:
# 여기서는 18 layers model 불러옴
# 4개의 layers에 basic blodk이 2개씩 총 8개의 blocks로 구성되어 있음
models.resnet18()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
### Model
resnet18 = models.resnet18(pretrained=True)

## Dataset
to_tensor = torchvision.transforms.Compose(
                [torchvision.transforms.ToTensor(),
               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]
                                          )

# cifar10 = torchvision.datasets.CIFAR10(root='./', download=True, transform=to_tensor)

# dataloader = torch.utils.data.DataLoader(cifar10, batch_size=8, shuffle=True, num_workers=2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 321MB/s]


100%|██████████| 170498071/170498071 [00:05<00:00, 28923974.47it/s]


Extracting ./cifar-10-python.tar.gz to ./


In [ ]:
for idx, data in enumerate(dataloader):

    img, gt = data

    print(img.shape)

    scores = resnet18(img)
    # print(scores)   # 추가
    print(scores.shape)
    break

torch.Size([8, 3, 32, 32])
torch.Size([8, 1000])


## Fine-tuning

In [ ]:
## TODO: 1. replace the last FC layer for cifar10
### Hint: 1000 -> 10
# import torch.nn as nn

num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 10)


In [ ]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# ## TODO: 2. fine tuning the last classifier (FC layer) using the cifar 10 training set.
# # train=True, False로 설정하면 train, test set을 알아서 다운로드 하는 듯함
# cifar10_train = datasets.CIFAR10(root='./', train=True, transform=to_tensor, download=True)
# cifar10_test = datasets.CIFAR10(root='./', train=False, transform=to_tensor, download=True)

# train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64)

# # 손실 함수와 옵티마이저 설정
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

# # 훈련 및 검증 함수 정의
# def train(model, dataloader, criterion, optimizer, num_epochs=10):
#     model.train()
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         for inputs, labels in dataloader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
#         print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(dataloader)}")

# def evaluate(model, dataloader):
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#     accuracy = correct / total
#     print(f"Accuracy on test set: {accuracy * 100:.2f}%")

# # 모델 훈련
# train(resnet18, train_loader, criterion, optimizer, num_epochs=10)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Loss: 1.0387940907569797
Epoch 2/10, Loss: 0.633057961378561
Epoch 3/10, Loss: 0.48273262791240307
Epoch 4/10, Loss: 0.3774539575819164
Epoch 5/10, Loss: 0.29434937591214316
Epoch 6/10, Loss: 0.22131955406397505
Epoch 7/10, Loss: 0.17738136760604656
Epoch 8/10, Loss: 0.13602025497256948
Epoch 9/10, Loss: 0.11572896346183079
Epoch 10/10, Loss: 0.09988525591652526


In [ ]:
# # accuracy와 tqdm 라이브러리로 progressive bar표시

# # import torch.optim as optim
# # from tqdm import tqdm  # tqdm 라이브러리 임포트

# # 데이터 전처리 및 데이터 로더 설정
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# train_dataset = datasets.CIFAR10(root='./', train=True, transform=transform, download=True)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# # 모델 정의 (예: ResNet-18)
# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)  # 출력 클래스 수를 10으로 변경

# # 손실 함수와 옵티마이저 설정
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # 에폭 수 설정
# num_epochs = 10

# # tqdm을 사용하여 훈련 진행률 표시
# for epoch in range(num_epochs):
#     model.train()  # 모델을 훈련 모드로 설정
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     # tqdm을 사용하여 진행 막대 표시
#     for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         _, predicted = outputs.max(1)
#         total += labels.size(0)
#         correct += predicted.eq(labels).sum().item()

#     # 에폭별로 훈련 손실과 정확도 출력
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {(100*correct/total):.2f}%')

# print('Finished Training')


In [ ]:
# # valid 추가
# # 검증 데이터셋 로딩 및 전처리 설정
# valid_dataset = datasets.CIFAR10(root='./', train=False, transform=transform, download=True)
# valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

# # 에폭마다 검증 수행
# for epoch in range(num_epochs):
#     model.train()  # 모델을 훈련 모드로 설정
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', leave=False):
#         # 훈련 스텝 (기존 코드와 동일)

#     # 훈련 손실과 정확도 출력 (기존 코드와 동일)

#     # 검증 단계 추가
#     model.eval()  # 모델을 평가 모드로 설정
#     valid_loss = 0.0
#     valid_correct = 0
#     valid_total = 0

#     with torch.no_grad():
#         for inputs, labels in tqdm(valid_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}', leave=False):
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             valid_loss += loss.item()

#             _, predicted = outputs.max(1)
#             valid_total += labels.size(0)
#             valid_correct += predicted.eq(labels).sum().item()

#     # 검증 결과 출력
#     print(f'Validation Loss: {valid_loss/len(valid_loader):.4f}, Validation Accuracy: {(100*valid_correct/valid_total):.2f}%')

# # 모델 훈련 완료 후에는 검증 결과를 사용하여 최종 성능을 평가하거나, 최적 모델을 저장할 수 있습니다.


In [ ]:
# # 위의 모든 것 모은 코드 + tensorboard + GPU 사용

# # import torch
# # import torch.nn as nn
# # import torch.optim as optim
# # from torchvision import datasets, transforms
# # from torchvision import models
# # from tqdm import tqdm
# # from tensorboardX import SummaryWriter  # TensorBoardX 임포트

# # CUDA를 사용할 수 있는지 확인하고 사용 가능한 GPU 디바이스 확인
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # 데이터 전처리 및 데이터 로더 설정
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # CIFAR-10 데이터셋 로딩
# train_dataset = datasets.CIFAR10(root='./', train=True, transform=transform, download=True)
# test_dataset = datasets.CIFAR10(root='./', train=False, transform=transform, download=True)

# # 데이터셋을 훈련 및 검증 데이터셋으로 나누기 (예: 80% 훈련, 20% 검증)
# train_size = int(0.8 * len(train_dataset))
# valid_size = len(train_dataset) - train_size
# train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

# # 데이터 로더 설정
# batch_size = 64
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # 모델 정의 (예: ResNet-18)
# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)  # 출력 클래스 수를 10으로 변경
# model.to(device)    # 모델을 GPU로 이동

# # 손실 함수와 옵티마이저 설정
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # 에폭 수 설정
# num_epochs = 10

# # TensorBoard 로그 디렉토리 설정
# writer = SummaryWriter()

# # (이하 코드는 이전 코드와 동일)

# # tqdm을 사용하여 훈련 진행률 표시
# for epoch in range(num_epochs):
#     model.train()  # 모델을 훈련 모드로 설정
#     running_loss = 0.0
#     correct = 0
#     total = 0

#   # 훈련 데이터셋에 대한 반복
#     for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
#         inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         _, predicted = outputs.max(1)
#         total += labels.size(0)
#         correct += predicted.eq(labels).sum().item()

#     # 에폭별로 훈련 손실과 정확도 출력
#     print(f'Training Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {(100*correct/total):.2f}%')

#     # TensorBoard에 로그 기록
#     writer.add_scalar('Training Loss', running_loss/len(train_loader), epoch)
#     writer.add_scalar('Training Accuracy', 100*correct/total, epoch)

#     # 검증 데이터셋에 대한 평가 (이하 코드는 이전 코드와 동일)

# # 모델 훈련 완료 후에는 TensorBoardX를 종료
# writer.close()

# print('Finished Training')

Using device: cuda
Using downloaded and verified file: ./cifar-10-python.tar.gz
Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training Epoch [1/10], Loss: 1.0903, Accuracy: 61.85%


Training Epoch [2/10], Loss: 0.6597, Accuracy: 77.13%


Training Epoch [3/10], Loss: 0.4884, Accuracy: 82.97%


...

Training Epoch [6/10], Loss: 0.2154, Accuracy: 92.60%


Training Epoch [7/10], Loss: 0.1662, Accuracy: 94.26%


Training Epoch [8/10], Loss: 0.1372, Accuracy: 95.19%


Training Epoch [9/10], Loss: 0.1242, Accuracy: 95.62%


Training Epoch [10/10], Loss: 0.0942, Accuracy: 96.78%
Finished Training


In [ ]:
# !tensorboard --logdir=./runs


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./runs


In [ ]:
## TODO: 2. fine tuning the last classifier (FC layer) using the cifar 10 training set.
# 위의 모든 것 모은 코드 + tensorboard(valid 관려 log도 추가) + GPU 사용

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torchvision import models
# from tqdm import tqdm
# from tensorboardX import SummaryWriter  # TensorBoardX 임포트

# CUDA를 사용할 수 있는지 확인하고 사용 가능한 GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터 전처리 및 데이터 로더 설정
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# CIFAR-10 데이터셋 로딩
train_dataset = datasets.CIFAR10(root='./', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./', train=False, transform=transform, download=True)

# 데이터셋을 훈련 및 검증 데이터셋으로 나누기 (예: 80% 훈련, 20% 검증)
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

# 데이터 로더 설정
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 (예: ResNet-18) 및 GPU로 이동
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 출력 클래스 수를 10으로 변경
model.to(device)    # 모델을 GPU로 이동

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 에폭 수 설정
num_epochs = 10

# TensorBoard 로그 디렉토리 설정
writer = SummaryWriter()

# tqdm을 사용하여 훈련 진행률 표시
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    # 훈련 데이터셋에 대한 반복
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # 에폭별로 훈련 손실과 정확도 출력
    print(f'Training Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {(100*correct/total):.2f}%')

    # TensorBoard에 훈련 손실과 정확도 로그 기록
    writer.add_scalar('Training Loss', running_loss/len(train_loader), epoch)
    writer.add_scalar('Training Accuracy', 100*correct/total, epoch)

    # 검증 데이터셋에 대한 평가
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    valid_correct = 0
    valid_total = 0

    # 검증 데이터셋에 대한 반복
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        valid_loss += loss.item()
        _, predicted = outputs.max(1)
        valid_total += labels.size(0)
        valid_correct += predicted.eq(labels).sum().item()

    # 검증 결과 출력
    print(f'Validation Loss: {valid_loss/len(valid_loader):.4f}, Validation Accuracy: {(100*valid_correct/valid_total):.2f}%')

    # TensorBoard에 검증 손실과 정확도 로그 기록
    writer.add_scalar('Validation Loss', valid_loss/len(valid_loader), epoch)
    writer.add_scalar('Validation Accuracy', 100*valid_correct/valid_total, epoch)

# 모델 훈련 완료 후에는 TensorBoardX를 종료
writer.close()

print('Finished Training')


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training Epoch [1/10], Loss: 1.0964, Accuracy: 61.83%
Validation Loss: 0.7710, Validation Accuracy: 73.74%


Training Epoch [2/10], Loss: 0.6571, Accuracy: 77.24%
Validation Loss: 0.6814, Validation Accuracy: 77.02%


Training Epoch [3/10], Loss: 0.4906, Accuracy: 82.67%
Validation Loss: 0.6568, Validation Accuracy: 77.96%


...

Training Epoch [6/10], Loss: 0.2163, Accuracy: 92.38%
Validation Loss: 0.7574, Validation Accuracy: 78.81%


Training Epoch [7/10], Loss: 0.1639, Accuracy: 94.23%
Validation Loss: 0.7988, Validation Accuracy: 78.91%


Training Epoch [8/10], Loss: 0.1427, Accuracy: 94.97%
Validation Loss: 0.8146, Validation Accuracy: 78.95%


Training Epoch [9/10], Loss: 0.1060, Accuracy: 96.34%
Validation Loss: 0.8693, Validation Accuracy: 79.31%


Training Epoch [10/10], Loss: 0.0892, Accuracy: 96.89%
Validation Loss: 0.9259, Validation Accuracy: 79.19%
Finished Training


In [ ]:
## TODO: 3. evaluation of the cifar 10 test set.
evaluate(resnet18, test_loader)

Accuracy on test set: 81.01%


In [ ]:
# 모델의 상태 및 가중치 저장
torch.save(model.state_dict(), 'best_model.pth')


In [ ]:
# 저장된 모델의 상태 및 가중치 불러오기
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

## Other models

In [ ]:
models.resnet50()

models.resnet101()

models.resnet152()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 